#### load test file

In [1]:
import json
json_file_path = './test_files.json'

# read test json file
with open(json_file_path) as f:
    test_data = json.load(f)

# # filter norm test data
test_files = [f for f in test_data if '*img.tif' in f]

In [2]:
# Define a function to recursively search for matching strings

import json
import fnmatch
def find_matching_strings(obj, pattern):
    matches = []
    if isinstance(obj, dict):
        for key, value in obj.items():
            matches.extend(find_matching_strings(value, pattern))
    elif isinstance(obj, list):
        for item in obj:
            matches.extend(find_matching_strings(item, pattern))
    elif isinstance(obj, str):
        if fnmatch.fnmatch(obj, pattern):
            matches.append(obj)
    return matches

# Define the pattern to match
pattern = '*img.tif'

# Find all matching strings in the JSON data
matching_strings = find_matching_strings(test_data, pattern)

In [3]:
test_id = [item.split('_')[0] for item in matching_strings]
test_id = sorted(test_id, key=int)

In [5]:
import pickle
with open('./shuffle_model.pkl', 'rb') as f:
    shuffle_model = pickle.load(f)

#### Check whether the saved image aligns with original image shuffle order

In [7]:
# you need to change path to your pkl path
with open('./cellpose_nonorm_all.pkl', 'rb') as f:
    cellpose_data = pickle.load(f)


In [8]:
with open('./stardist_all.pkl', 'rb') as f:
    stardist_data = pickle.load(f)

In [10]:
import torch
import glob
### swinunetr_2D
swin2d_dir = '/mnt/aperto/yin/swinunetr_results/swinunetr_2D/' 
swin2d_files = sorted(glob.glob(f'{swin2d_dir}*_img_pred.pt'))
swin2d_predict = {}
for item in swin2d_files:
    idx = item.split('/')[-1].split('_')[0]
    swin2d_predict[idx] = torch.load(item)['instance_mask']

/tmp/ipykernel_421104/2055031262.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  swin2d_predict[idx] = torch.load(item)['instance_mask']


In [11]:
### swinunetr_3D_window3
swin_3D_pretrain_window3_dir = '/mnt/aperto/yin/swinunetr_results/swinunetr_3D_pretrain_window3/' 
swin_3D_pretrain_window3_files = sorted(glob.glob(f'{swin_3D_pretrain_window3_dir}*_img_pred.pt'))
swin_3D_pretrain_window3_predict = {}
for item in swin_3D_pretrain_window3_files:
    idx = item.split('/')[-1].split('_')[0]
    swin_3D_pretrain_window3_predict[idx] = torch.load(item)['instance_mask']

/tmp/ipykernel_421104/750914079.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  swin_3D_pretrain_window3_predict[idx] = torch.load(item)['instance_mask']


In [12]:
models_map = {
    1:cellpose_data,
    2:stardist_data,
    3:swin2d_predict,
    4:swin_3D_pretrain_window3_predict
}

In [16]:
import tiffile as tf
import numpy as np
for img_id in test_id:
    model1_img = tf.imread(f'./crops/{img_id}_model1.tif')
    model2_img = tf.imread(f'./crops/{img_id}_model2.tif')
    model3_img = tf.imread(f'./crops/{img_id}_model3.tif')
    model4_img = tf.imread(f'./crops/{img_id}_model4.tif')
    model_ori1 = models_map[shuffle_model[img_id][0]][img_id]
    model_ori2 = models_map[shuffle_model[img_id][1]][img_id]
    model_ori3 = models_map[shuffle_model[img_id][2]][img_id]
    model_ori4 = models_map[shuffle_model[img_id][3]][img_id]
    are_equal1 = np.array_equal(model1_img, model_ori1)
    are_equal2 = np.array_equal(model2_img, model_ori2)
    are_equal3 = np.array_equal(model3_img, model_ori3)
    are_equal4 = np.array_equal(model4_img, model_ori4)
    if are_equal1 and are_equal2 and are_equal3 and are_equal4:
        print(img_id, 'same')
    

0005 same
0006 same
0009 same
0017 same
0022 same
0024 same
0030 same
0033 same
0042 same
0045 same
0046 same
0056 same
0059 same
0060 same
0063 same
0066 same
0075 same
0077 same
0079 same
0084 same
0090 same
0101 same
0109 same
0113 same
0117 same
0118 same
0119 same
0124 same
0125 same
0143 same
0146 same
0147 same
0158 same
0159 same
0165 same
0168 same
0172 same
0176 same
0177 same
0181 same
0183 same
0202 same
0213 same
0221 same
0227 same
0231 same
0238 same
0244 same
0246 same
0260 same
0269 same
0271 same
0274 same
0275 same
0277 same
0279 same
0281 same
0282 same
0285 same
